In [50]:
!pip install pinecone
!pip install langchain-pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
!pip install langchain_groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.
langchain-pinecone 0.2.0 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.2.43 which is incompatible.


In [48]:
import os
import time
from dotenv import load_dotenv
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# from pinecone import Pinecone
from langchain_pinecone import Pinecone
from pinecone import ServerlessSpec
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

In [49]:
from langchain_groq import ChatGroq

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.9/u/custom-json-schema

In [30]:
api_key = os.getenv("PINECONE_API_KEY")

In [31]:
##Read the Document 
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents


In [32]:
doc=read_doc('uploads/')
doc
len(doc)

1

In [33]:
# cHUNKING Of The Data
def chunk_data(docs,chunk_size=800,chunk_overlap=80):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunked_docs=text_splitter.split_documents(docs)
    return chunked_docs

In [34]:
chunked_docs=chunk_data(doc)
chunked_docs

[Document(metadata={'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf', 'page': 0}, page_content='SL.NO. CANDIDATE NAME GENDER COURSE STREAM INSTITUTE NAME EMAIL ADDRESS CONTACT NO.\n1 SUBRAT SETHY Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE subratsethy017@gmail.com 8658132036\n2 ABHIJIT SAMANT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE lukshi007@gmail.com 7735099393\n3 SHUBHAM PRADHAN Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE shubhampradhan813@gmail.com 8770367753\n4 ABHIJIT MANSINGH Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE abhijitmsing08@gmail.com 8926236278\n5 AYAN KANUNGO Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE aayankanungo@gmail.com 9437851770\n6 ANSUMAN SIMANTA SEKHAR BHUJABALA Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ansumanbhujabal1@gmail.com 8260964834'),
 Document(metadata={'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 

In [35]:
len(chunked_docs)

3

In [36]:
embeddings = HuggingFaceEmbeddings()

In [37]:
vectors=embeddings.embed_documents("How is me ?")
len(vectors)

11

In [38]:
from pinecone import Pinecone


In [39]:

pc = Pinecone(api_key=api_key)
index_name="budget"

In [40]:
from langchain_pinecone import Pinecone

In [41]:
index = pc.Index(index_name)
# wait a moment for connection
time.sleep(1)

index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [42]:
list=pc.list_indexes()

In [15]:
print(list)

[{
    "name": "budget",
    "dimension": 768,
    "metric": "cosine",
    "host": "budget-ywnft4n.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}, {
    "name": "langchain-test-index",
    "dimension": 3072,
    "metric": "cosine",
    "host": "langchain-test-index-ywnft4n.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "deletion_protection": "disabled"
}]


In [43]:
print(list[0].name)

budget


In [44]:
if index_name  == list[0].name:
    # pinecone.create_index(
    #     name=index_name, 
    #     metric="cosine",
    #     dimension=DIMENSIONS)
    # docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)
    print("Please Create Index first")
else:
     print("Please Create Index 2nd")
    # docsearch = pc.from_existing_index(doc,embeddings,index_name=index_name)

Please Create Index first


In [45]:

index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [46]:
llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0
    )
chain=load_qa_chain(llm,chain_type="stuff")


NameError: name 'ChatGroq' is not defined

In [23]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [24]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [21]:
our_query = "Tell me the student names  "
answer = retrieve_answers(our_query)
print(answer)

NameError: name 'index' is not defined

In [28]:
our_query = "Tell me  about the student ANSUMAN  "
answer = retrieve_answers(our_query)
print(answer)

[Document(id='f08a75a8-4be8-4687-a0ad-e0b84eb6de3f', metadata={'page': 0.0, 'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf'}, page_content='13 UJJWAL KUMAR ROUT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ujjwalkumarr5@gmail.com 7978740370\n14 DURGADEVI34@GMAIL.COM Female B.E. / B.TECH EACE PARALA MAHARAJA ENGINEERING COLLEGE samantaradurgadevi34@gmail.com 7653912759\n15 B . RACHANA PATRA Female B.E. / B.TECH ETC PARALA MAHARAJA ENGINEERING COLLEGE rachanapatra07@gmail.com 8260439167\n16 SUBHAM KUMAR PADHY Male B.E. / B.TECH ETC PARALA MAHARAJA ENGINEERING COLLEGE subhamkumarpadhy14@gmail.com 6370293866\n17 CHAKRA NARAYANA MISHRA Male B.E. / B.TECH ETC PARALA MAHARAJA ENGINEERING COLLEGE chakramishra6@gmail.com 8260699385\n18 AKHILESH MISRA Male B.E. / B.TECH ETC PARALA MAHARAJA ENGINEERING COLLEGE akhileshmisra2003@gmail.com 7978485388'), Document(id='91a3f85b-1a26-4275-8ca7-5625d4cbf2a3', metadata={'page': 0.0, 'source': 'uploads/Eligi